<a href="https://colab.research.google.com/github/Amir-D-Shadow/A-study-of-pattern-between-the-academic-behaviour-of-students/blob/main/Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from numba import cuda,float64,int64
import numpy as np
import math

In [ ]:
@cuda.jit
def func(A,B,W,b,stride,Hlim,Wlim,Clim):

  x = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
  y = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
  z = cuda.threadIdx.z +